In [0]:
import os
from openai import AzureOpenAI
    



client = AzureOpenAI(
    api_version="2024-08-01-preview",
    
    azure_endpoint="PLACEHOLDER",
    api_key="PLACEHOLDER",
)

In [0]:
client_v2 = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="PLACEHOLDER",
    api_key="PLACEHOLDER"
)

In [0]:
def callmodel4o(deployment_name, prompt):

  start_phrase = prompt
  response = client.chat.completions.create(
      model=deployment_name,  # This refers to the deployed model name (gpt-4o)
      messages=[
          {"role": "system", "content": "You are a code converting specialist between SAP Code and Databricks code."},
          {"role": "user", "content": start_phrase}
      ],
      max_tokens=10000,  # Set the maximum length of the response
      temperature=0.2,  # Lower temperature for more deterministic responses
      top_p=0.8,        # More standard responses
      frequency_penalty=0.1, # Allow code repetition
      presence_penalty=0.0 # Less varied content
  )



  #print(response)
  print('---------------------------------------------------')
  print(response.choices[0].message.content)
  return response.choices[0].message.content

In [0]:
def callmodelo1(deployment_name, prompt):

  start_phrase = prompt
  response = client_v2.chat.completions.create(
      model=deployment_name,  # This refers to the deployed model name (gpt-4o)
      messages=[
        {
            "role": "user",
            "content": start_phrase
        }
    ]
  )



  #print(response)
  print('---------------------------------------------------')
  print(response.choices[0].message.content)
  return response.choices[0].message.content


In [0]:
def call_model_o1(deployment_name, prompt):
    # Define the start phrase as the user input (XML configuration and prompt)
    start_phrase = prompt  # The prompt you want to send to the model

    # Define the system message that provides the context
    system_message = {
        "role": "system",
        "content": "You are an expert in converting SAP logic, represented in XML configuration, into Databricks SQL. " +
                   "Your task is to parse the logic in the XML and generate the corresponding Databricks SQL queries."
    }

    # Make the API call to get the response from the model
    response = client_v2.chat.completions.create(
        model=deployment_name,  # The name of your deployed model (e.g., gpt-4-deployment)
        messages=[
            system_message,  # Add the system message to set the context
            {
                "role": "user",  # User is sending the message
                "content": start_phrase  # The actual prompt content (XML configuration)
            }
        ]
    )
    
    # Print the response for debugging purposes
    print('---------------------------------------------------')
    
    # Correctly access the content using dot notation
    print(response.choices[0].message.content)  # Use .content to access the message content
    
    return response.choices[0].message.content  # Return the model's response

In [0]:

if 'is_target_notebook' not in globals():
    is_target_notebook = False

# Only create widgets if we're in the target notebook
if is_target_notebook:
    # Create a text widget for manually inserting the directory name
    dbutils.widgets.text("data_flow_insertion", "", "Enter Directory Name (e.g., DF1)")

    # Create a dropdown widget for selecting from a list of DF1, DF2, ..., DF48
    df_list = [f"DF{i}" for i in range(1, 49)]  # List of DF1, DF2, ..., DF48
    dbutils.widgets.dropdown("data_flow_selection", df_list[0], df_list, "Select Directory")
else:
    print("No widgets created in the source notebook")

In [0]:
def read_file_content(file_path):
    """Read the content of a file using Spark's higher-level API (spark.read.text)."""
    try:
        # Read the file content using spark.read.text
        df = spark.read.text(file_path)
        file_content = df.rdd.map(lambda r: r[0]).collect()
        return file_content
    except Exception as e:
        log_message = f"Error reading file: {file_path}\nException: {e}"
        log_messages.append(f"**Read Error**:\n{log_message}\n") # Store log message with section heading
        print(log_message) # Print to console
        return None

def log_to_blob(message, action):
    """Store log messages and print them to the console."""
    try:
        # Print to the console
        print(message)
        # Store log message in the list
        log_messages.append(f"**{action}**:\n{message}\n")
    except Exception as e:
        print(f"Error logging: {e}")